<a href="https://colab.research.google.com/github/mapaloCat/Traffic-Sign-Detection-System/blob/master/03_traffic_sign_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The German Traffic Sign Benchmark

Collaborator 1: Panagiotis Michalopoulos

Collaborator 2: Filip Finfando

Download full data set from http://benchmark.ini.rub.de/?section=gtsdb&subsection=dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!wget -c https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip
!unzip FullIJCNN2013.zip

In [0]:
import numpy as np
import cv2

IMG_HEIGHT = 600
SIGN_SIZE = (96, 96)

# Function for reading the images
def readImages(rootpath, images_range, signs_range, datapath, scaled):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.
    Arguments: path to the traffic sign data, for example 'FullIJCNN2013'
    Returns:   list of images, list of corresponding labels'''
    images = {} # original image
    scales = {} # original scale
    for num in images_range:
        filename = rootpath + '/' + "{:05d}".format(num) + '.ppm'
        img = cv2.imread(filename, cv2.IMREAD_COLOR)
        scale = IMG_HEIGHT / float(img.shape[0])
        img_resized = cv2.resize(img, (int(img.shape[1]*scale),int(img.shape[0]*scale)))
        images.setdefault(filename,[]).append(img_resized)
        scales.setdefault(filename,[]).append(scale)

    files = [] # filenames
    signs = [] # traffic sign image
    bboxes = [] # corresponding box detection
    labels = [] # traffic sign type
    data = np.genfromtxt(datapath, delimiter=';', dtype=str, usecols=range(0, 6))
    for elem in signs_range:
        filename = rootpath + '/' + data[elem][0]
        img = images.get(filename)[0]
        if scaled:
          scale = 1
        else:
          scale = scales.get(filename)[0]
        bbox = np.array([int(data[elem][1]), int(data[elem][2]), int(data[elem][3]), int(data[elem][4])]) * scale
        sign = img[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        sign_resized = cv2.resize(sign, SIGN_SIZE)
        files.append(filename)
        signs.append(sign_resized)
        bboxes.append(bbox)
        labels.append(data[elem][5])
    return images, files, signs, bboxes, labels

In [0]:
# The German Traffic Sign Recognition Benchmark
train_images, train_files, train_signs, train_bboxes, train_labels = readImages('FullIJCNN2013', range(0,600), range(0,852), 'FullIJCNN2013/gt.txt', False)
test_images, test_files, test_signs, test_bboxes, test_labels = readImages('FullIJCNN2013', range(600,900), range(852,1213), 'FullIJCNN2013/gt.txt', False)

## Traffic sign detection

In [0]:
import pickle

with open('drive/My Drive/models/train_pred.pickle', 'rb') as fp:
    train_pred = pickle.load(fp)
with open('drive/My Drive/models/test_pred.pickle', 'rb') as fp:
    test_pred = pickle.load(fp)

In [0]:
## import binary model

from keras.models import model_from_json
# load json and create model
json_file = open('drive/My Drive/models/bin_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cnn = model_from_json(loaded_model_json)
# load weights into new model
cnn.load_weights("drive/My Drive/models/bin_model.h5")
print("Loaded binary model from disk")

In [0]:
## import classification model

# load json and create model
json_file = open('drive/My Drive/models/classification_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cnn_classifier = model_from_json(loaded_model_json)
# load weights into new model
cnn_classifier.load_weights("drive/My Drive/models/classification_model.h5")
print("Loaded classification model from disk")

In [0]:
# import matplotlib.pyplot as plt
# import math

# # run some tests
# i=0

# number_of_images = 20
# number_of_boxes_to_draw = 1000


# for filename in test_images:
#     # Draw predictions
#     aux = test_images.get(filename)[0].copy()
#     print(filename)
#     #print(test_labels[i])
    
#     counter = 0
    
#     for bbox in test_pred.get(filename)[0][:number_of_boxes_to_draw]: 
        
#         #filter some boxes before making predictions
#         dx = int(bbox[0])-int(bbox[2])
#         dy = int(bbox[1])-int(bbox[3])
#         ratio = abs(dy/dx)
#         if ratio<1:
#             ratio=1/ratio
#         if ratio>2 or abs(dx)>96 or abs(dy)>96:
#             continue
        
#         # make a prediction
#         roi = test_images.get(filename)[0][int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
#         SIGN_SIZE_CNN = (96, 96)
#         roi_resized = cv2.resize(roi, SIGN_SIZE_CNN)
#         roi_resized = roi_resized.astype('float32')
#         roi_resized /= 255.0
#         roi_resized = np.reshape(roi_resized, [1,SIGN_SIZE_CNN[0],SIGN_SIZE_CNN[1],3])
        
#         confidence = cnn.predict(roi_resized)
#         counter+=1
#         #print(counter, " confidence: ", confidence, "\n")
        
#         sign_class = cnn_classifier.predict(roi_resized)
        
#         # print rectangle if confidence is large
#         # those are boxes that we think have the largest probability to have a traffic sign inside
#         if confidence[0][1] >= 0.5:
#             cv2.rectangle(aux, (int(bbox[0]),int(bbox[1])), (int(bbox[2]),int(bbox[3])), (0,255,0), 3)
#             #print(counter, " sign_class:", str(np.argmax(sign_class[0])), " diagonal:", math.sqrt(dx**2 + dy**2), "\n")
            
            
#         #print(bbox,"\n")
#             #sign_class_int = np.where(sign_class[0]==sign_class[0].max())[0][0]
# #             print(sign_class_int)



#         # print contents of box if you like
# #         not_a_sign = aux[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
# #         plt.imshow(cv2.cvtColor(not_a_sign, cv2.COLOR_BGR2RGB))
# #         plt.show()
            

    
#     # Draw ground truth
#     # this is what we should detect
#     if filename in test_files:
#         for idx in [i for i, x in enumerate(test_files) if x == filename]:
#             bbox = test_bboxes[idx]
#             #print(bbox)
#             dx = int(bbox[0])-int(bbox[2])
#             dy = int(bbox[1])-int(bbox[3])
#             ratio = abs(dy/dx)
# #             if ratio<1:
# #                 print(1/ratio)
# #             else:
# #                 print(ratio)
#             print(bbox)
#             cv2.rectangle(aux, (int(bbox[0]),int(bbox[1])), (int(bbox[2]),int(bbox[3])), (0,0,255), 3)
            
#             # print contents of box if you like
# #             sign = aux[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
# #             plt.imshow(cv2.cvtColor(sign, cv2.COLOR_BGR2RGB))
# #             plt.show()
    
#     plt.imshow(cv2.cvtColor(aux, cv2.COLOR_BGR2RGB))
#     plt.show()
#     i+=1
#     if i==number_of_images: break

### Evaluation using Mean Average Precision

git clone https://github.com/Cartucho/mAP.git

mkdir ground-truth

mkdir predicted

In [0]:
!git clone https://github.com/Cartucho/mAP.git

In [0]:
# !mkdir ground-truth

# !mkdir predicted

In [0]:
!ls

In [0]:
!cd mAP/input/ground-truth; \rm -rf 2007_*

In [0]:
!cd mAP/input/detection-results; \rm -rf *

In [0]:
for filename in test_images:
    anns_ofs = open('mAP/input/ground-truth/' + filename[-9:-4:] + '.txt', 'w')
    if filename in test_files:
        for idx in [i for i, x in enumerate(test_files) if x == filename]:
            bbox = test_bboxes[idx]
            label = test_labels[idx]
            anns_ofs.write(str(label) + ' ' + str(bbox[0]) + ' ' + str(bbox[1]) + ' ' + str(bbox[2]) + ' ' + str(bbox[3]) + '\n')
    anns_ofs.close()

In [0]:
import matplotlib.pyplot as plt
from time import time

start = time()
for filename in test_images:
    prds_ofs = open('mAP/input/detection-results/' + filename[-9:-4:] + '.txt', 'w')
    aux = test_images.get(filename)[0].copy()
    
    for bbox in test_pred.get(filename)[0]:
        
        #filter some boxes before making predictions
        dx = int(bbox[0])-int(bbox[2])
        dy = int(bbox[1])-int(bbox[3])
        ratio = abs(dy/dx)
        if ratio<1:
          ratio=1/ratio
        if ratio>2 or abs(dx)>96 or abs(dy)>96:
          continue
        
        roi = test_images.get(filename)[0][int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        SIGN_SIZE_CNN = (96, 96)
        roi_resized = cv2.resize(roi, SIGN_SIZE_CNN)
        roi_resized = roi_resized.astype('float32')
        roi_resized /= 255.0
        roi_resized = np.reshape(roi_resized, [1,SIGN_SIZE_CNN[0],SIGN_SIZE_CNN[1],3])
    
        confidence = cnn.predict(roi_resized)
        if confidence[0][1] >= 0.5:
          cv2.rectangle(aux, (int(bbox[0]),int(bbox[1])), (int(bbox[2]),int(bbox[3])), (0,0,255), 3)
        else:
          continue
                                                
        ## sign class
        sign_class = cnn_classifier.predict(roi_resized)
#         sign_class_int = np.where(sign_class[0]==sign_class[0].max())[0][0]
#         print(sign_class_int)
        
        
        # traffic_sign_class confidence bbox
        prds_ofs.write(str(np.argmax(sign_class[0])) + ' ' + str(confidence[0][1]) + ' ' + str(bbox[0]) + ' ' + str(bbox[1]) + ' ' + str(bbox[2]) + ' ' + str(bbox[3]) + '\n')
    prds_ofs.close()
    plt.imshow(cv2.cvtColor(aux, cv2.COLOR_BGR2RGB))
    plt.show()
end = time()
print("Traffic sign detection took " + str(end - start) + " seconds")

In [0]:
!ls

In [0]:
# Follow previous mAP code in order to evaluate the performance of your neural net
%run mAP/main.py -na -np
%cd ../

In [0]:
# Sample results obtained using our detector
81.53% = 1 AP 
86.04% = 10 AP 
71.55% = 11 AP 
83.11% = 12 AP 
88.96% = 13 AP 
96.67% = 14 AP 
100.00% = 15 AP 
100.00% = 16 AP 
69.17% = 17 AP 
59.09% = 18 AP 
85.71% = 2 AP 
62.50% = 22 AP 
72.62% = 23 AP 
100.00% = 24 AP 
65.00% = 25 AP 
41.90% = 26 AP 
60.00% = 28 AP 
0.00% = 29 AP 
64.44% = 3 AP 
66.67% = 30 AP 
0.00% = 31 AP 
45.00% = 32 AP 
100.00% = 33 AP 
66.67% = 34 AP 
92.50% = 35 AP 
0.00% = 36 AP 
100.00% = 37 AP 
63.75% = 38 AP 
50.00% = 39 AP 
83.70% = 4 AP 
66.67% = 40 AP 
100.00% = 41 AP 
90.00% = 42 AP 
68.98% = 5 AP 
100.00% = 6 AP 
95.00% = 7 AP 
95.00% = 8 AP 
82.41% = 9 AP 
mAP = 72.49%

Traffic sign detection took 520.6455109119415 seconds